In [35]:
pip install aiofiles==24.1.0 annotated-types==0.7.0 anthropic==0.69.0 anyio==4.11.0 Brotli==1.1.0 certifi==2025.8.3 charset-normalizer==3.4.3 click==8.3.0 distro==1.9.0 docstring_parser==0.17.0 fastapi==0.118.0 ffmpy==0.6.1 filelock==3.19.1 fsspec==2025.9.0 gradio==5.49.0 gradio_client==1.13.3 greenlet==3.2.3 groovy==0.1.2 h11==0.16.0 hf-xet==1.1.10 httpcore==1.0.9 httpx==0.28.1 huggingface-hub==0.35.3 idna==3.10 Jinja2==3.1.6 jiter==0.11.0 markdown-it-py==4.0.0 MarkupSafe==3.0.3 mdurl==0.1.2 numpy==2.3.3 openai==2.1.0 orjson==3.11.3 packaging==25.0 pandas==2.3.3 pillow==11.3.0 playwright==1.54.0 pydantic==2.11.10 pydantic_core==2.33.2 pydub==0.25.1 pyee==13.0.0 Pygments==2.19.2 python-dateutil==2.9.0.post0 python-multipart==0.0.20 pytz==2025.2 PyYAML==6.0.3 requests==2.32.5 rich==14.1.0 ruff==0.13.3 safehttpx==0.1.6 semantic-version==2.10.0 shellingham==1.5.4 six==1.17.0 sniffio==1.3.1 starlette==0.48.0 tomlkit==0.13.3 tqdm==4.67.1 typer==0.19.2 typing-inspection==0.4.2 typing_extensions==4.14.1 tzdata==2025.2 urllib3==2.5.0 uvicorn==0.37.0 websockets==15.0.1

Note: you may need to restart the kernel to use updated packages.


In [36]:
# Import required libraries
import gradio as gr
import openai
import anthropic
import os
import sys
import io
import time
import subprocess
import shutil
from typing import Dict, List, Tuple

In [37]:
# Define constants, prompts, and example Python code
TRANSLATE_SYSTEM_PROMPT = """You are an expert at translating Python code to highly optimized C++. Focus on:
Using efficient data types, applying compiler optimizations, minimizing memory allocations, and using appropriate C++ standard library features.
Generate only the C++ code with necessary includes, no explanations."""

EXPLAIN_SYSTEM_PROMPT_TEMPLATE = """You are a helpful coding mentor. You have access to the original Python code and the translated C++ code. Your task is to answer user questions about the code, optimizations, performance, or C++ concepts. Be clear, friendly, and use code snippets in your explanations when helpful.

**Original Python Code:**
```python
{python_code}
'''

**Translated C++ Code:**
```cpp
{cpp_code}
```
"""

In [38]:

DEFAULT_PYTHON_CODE = {
    "Calculate Pi": """import time

def calculate_pi(iterations):
    result = 1.0
    for i in range(1, iterations + 1):
        j = i * 4 - 1
        result -= (1.0 / j)
        j = i * 4 + 1
        result += (1.0 / j)
    return result * 4

start = time.time()
pi = calculate_pi(10_000_000)
end = time.time()

print(f"Pi: {pi:.10f}")
print(f"Time: {end - start:.6f}s")""",
    "Fibonacci": """import time

def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

start = time.time()
result = fibonacci(35)
end = time.time()

print(f"Fibonacci(35): {result}")
print(f"Time: {end - start:.6f}s")""",
    "Matrix Multiplication": """import time

def matrix_multiply(size):
    A = [[i + j for j in range(size)] for i in range(size)]
    B = [[i - j for j in range(size)] for i in range(size)]
    C = [[0] * size for _ in range(size)]
    
    for i in range(size):
        for j in range(size):
            for k in range(size):
                C[i][j] += A[i][k] * B[k][j]
    return C

start = time.time()
result = matrix_multiply(200)
end = time.time()

print(f"Matrix 200x200 multiplied")
print(f"Time: {end - start:.6f}s")"""
}

In [39]:
def extract_code_from_markdown(text: str) -> str:
    """Extract code from markdown code blocks if present"""
    text = text.strip()
    
    # Check if wrapped in code blocks
    if text.startswith("```"):
        lines = text.split('\n')
        # Remove first line (```cpp or ```c++)
        lines = lines[1:]
        # Remove last line if it's ```
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        return '\n'.join(lines)
    
    return text

In [40]:


# Function to update API settings
def update_api_settings(provider: str, key: str, settings: Dict) -> Tuple[Dict, gr.Button, gr.Button, gr.Button]:
    if not key:
        raise gr.Error("Please enter your API key in the Settings tab.")
    if provider == "OpenAI" and not key.startswith("sk-"):
        raise gr.Error("Invalid OpenAI key. It should start with 'sk-'.")
    
    settings["provider"] = provider
    settings["key"] = key
    gr.Info(f"{provider} API key updated successfully!")
    return settings, gr.Button(interactive=True), gr.Button(interactive=True), gr.Button(interactive=True)

In [41]:
# Function to translate Python code to C++ using API
def translate_code(python_code: str, settings: Dict, progress=gr.Progress(track_tqdm=True)) -> str:
    provider, api_key = settings.get("provider"), settings.get("key")
    if not api_key:
        raise gr.Error("API key is not set. Please go to the ⚙️ Settings tab to enter it.")
    if not python_code.strip():
        raise gr.Error("Please enter some Python code to translate.")

    progress(0, desc="Connecting to API...")
    try:
        if provider == "OpenAI":
            client = openai.OpenAI(api_key=api_key)
            progress(0.5, desc="Translating with GPT-4...")
            response = client.chat.completions.create(model="gpt-4", messages=[{"role": "system", "content": TRANSLATE_SYSTEM_PROMPT}, {"role": "user", "content": python_code}], temperature=0.1)
            return response.choices[0].message.content.strip()
        else:  # Claude
            client = anthropic.Anthropic(api_key=api_key)
            progress(0.5, desc="Translating with Claude Sonnet...")
            response = client.messages.create(model="claude-sonnet-4-20250514", max_tokens=4096, system=TRANSLATE_SYSTEM_PROMPT, messages=[{"role": "user", "content": python_code}])
            return response.content[0].text.strip()
    except openai.AuthenticationError:
        raise gr.Error("OpenAI authentication failed. Please check your API key.")
    except anthropic.AuthenticationError:
        raise gr.Error("Claude authentication failed. Please check your API key.")
    except Exception as e:
        raise gr.Error(f"An unexpected API error occurred: {e}")

In [42]:
# Function to run Python code and capture output and execution time
def run_python_code(code: str) -> Tuple[str, float]:
    output_capture = io.StringIO()
    original_stdout, sys.stdout = sys.stdout, output_capture
    start_time = time.perf_counter()
    try:
        compile(code, "<string>", "exec")
        exec(code, {})
    except SyntaxError as e:
        sys.stdout = original_stdout
        return f"Python Syntax Error:\n{e}", 0.0
    except Exception as e:
        sys.stdout = original_stdout
        return f"Error executing Python code:\n{e}", 0.0
    finally:
        sys.stdout = original_stdout
    end_time = time.perf_counter()
    return output_capture.getvalue(), end_time - start_time

In [43]:
# Function to compile and run C++ code
def run_cpp_code(code: str) -> Tuple[str, float]:
    print("=== CHECKING G++ ===")
    compiler_path = shutil.which("g++")
    print(f"G++ found at: {compiler_path}")
    
    if not compiler_path:
        raise gr.Error("C++ compiler (g++) not found. Please install MinGW and add it to your system's PATH.")
    
    source_path, exe_path = "optimized.cpp", "optimized.exe"
    
    print(f"=== WRITING C++ FILE ===")
    print(f"First 200 chars of code:\n{code[:200]}")
    
    with open(source_path, "w", encoding="utf-8") as f:
        f.write(code)
    
    print("=== COMPILING ===")
    compile_cmd = ["g++", "-Ofast", "-std=c++17", "-march=native", "-o", exe_path, source_path]
    print(f"Compile command: {' '.join(compile_cmd)}")
    
    compile_proc = subprocess.run(compile_cmd, capture_output=True, text=True, timeout=15)
    
    print(f"Compilation return code: {compile_proc.returncode}")
    if compile_proc.stderr:
        print(f"Compilation stderr (first 500 chars):\n{compile_proc.stderr[:500]}")
    
    if compile_proc.returncode != 0:
        error_message = f"Compilation failed with {len(compile_proc.stderr.splitlines())} errors.\nSee details below:\n\n{compile_proc.stderr}"
        if os.path.exists(source_path):
            os.remove(source_path)
        return error_message, 0.0

    start_time = time.perf_counter()
    try:
        run_proc = subprocess.run([os.path.abspath(exe_path)], capture_output=True, text=True, timeout=10, check=True)
        output = run_proc.stdout
    except subprocess.TimeoutExpired:
        return "Execution timed out after 10 seconds.", 0.0
    except subprocess.CalledProcessError as e:
        return f"Execution failed with return code {e.returncode}:\n{e.stderr}", 0.0
    finally:
        end_time = time.perf_counter()
        if os.path.exists(source_path): os.remove(source_path)
        if os.path.exists(exe_path): os.remove(exe_path)
        
    return output, end_time - start_time

In [44]:
# Function to run and compare Python and C++ code
def run_and_compare(python_code: str, cpp_code: str, progress=gr.Progress(track_tqdm=True)) -> str:
    if not python_code.strip() or not cpp_code.strip():
        raise gr.Error("Both Python and C++ code must be present to run a comparison.")
    
    print("=== DEBUG: Python code ===")
    print(python_code[:200])  # Print first 200 chars
    print("=== DEBUG: C++ code ===")
    print(cpp_code[:200])  # Print first 200 chars
    
    progress(0, desc="Running Python code...")
    py_out, py_time = run_python_code(python_code)
    
    print(f"=== DEBUG: Python output ===")
    print(py_out)
    print(f"Python time: {py_time}")
    
    progress(0.5, desc="Compiling & Running C++ code...")
    cpp_out, cpp_time = run_cpp_code(cpp_code)
    
    print(f"=== DEBUG: C++ output ===")
    print(cpp_out)
    print(f"C++ time: {cpp_time}")

    py_result = f"PYTHON OUTPUT:\n{py_out.strip()}\n\nExecution Time: {py_time:.6f}s"
    cpp_result = f"C++ OUTPUT:\n{cpp_out.strip()}\n\nExecution Time: {cpp_time:.6f}s"
    
    divider = "═" * 50
    speedup_msg = "Could not determine speedup due to errors or zero execution time."
    if py_time > 0 and cpp_time > 0 and "Error" not in cpp_out and "failed" not in cpp_out:
        speedup = py_time / cpp_time
        speedup_msg = f"SPEEDUP: C++ was {speedup:.2f}x faster"

    return f"{divider}\n{py_result}\n{divider}\n{cpp_result}\n{divider}\n{speedup_msg}"

In [45]:
def explain_code(chat_history: List[Dict[str, str]], user_question: str, settings: Dict, py_code: str, cpp_code: str, progress=gr.Progress(track_tqdm=True)):
    provider, api_key = settings.get("provider"), settings.get("key")
    if not api_key:
        raise gr.Error("API key is not set. Please configure it in the ⚙️ Settings tab.")
    if not user_question.strip():
        return chat_history, ""
    if not py_code.strip() or not cpp_code.strip():
        raise gr.Error("Please translate the code first before asking questions.")
    progress(0, desc="Thinking...")
    
    chat_history.append({"role": "user", "content": user_question})
    system_prompt = EXPLAIN_SYSTEM_PROMPT_TEMPLATE.format(python_code=py_code, cpp_code=cpp_code)
    try:
        if provider == "OpenAI":
            client = openai.OpenAI(api_key=api_key)
            messages_for_api = [{"role": "system", "content": system_prompt}] + chat_history
            response = client.chat.completions.create(model="gpt-4", messages=messages_for_api, temperature=0.5)
            bot_message = response.choices[0].message.content
        else:  # Claude
            client = anthropic.Anthropic(api_key=api_key)
            # Claude needs clean messages without system role in the array
            # Make sure chat_history only has user/assistant messages
            clean_history = [msg for msg in chat_history if msg.get("role") in ["user", "assistant"]]
            response = client.messages.create(
                model="claude-sonnet-4-20250514", 
                max_tokens=4096, 
                system=system_prompt, 
                messages=clean_history
            )
            bot_message = response.content[0].text
            
        chat_history.append({"role": "assistant", "content": bot_message})
        return chat_history, ""
    except Exception as e:
        raise gr.Error(f"An API error occurred: {e}")

In [46]:
# Function to load example Python code
def load_example(example_name: str) -> str:
    return DEFAULT_PYTHON_CODE.get(example_name, "")

In [47]:
# Set up Gradio UI and event handlers
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="sky"), title="Code Translator") as demo:
    app_settings = gr.State({"provider": "OpenAI", "key": None})
    python_code_state = gr.State(DEFAULT_PYTHON_CODE["Calculate Pi"])
    cpp_code_state = gr.State("")

    gr.Markdown("# 🤖 Python to C++ Code Translator & Performance Analyzer")
    gr.Markdown("A  tool to translate Python code to optimized C++, execute both versions, compare their performance, and explain the optimizations.")
    
    with gr.Accordion("Expand for Instructions", open=False):
        gr.Markdown("""
        ### How to Use This Tool
        1.  **⚙️ Settings:** Choose your AI provider (OpenAI or Claude) and enter your API key. The buttons on other tabs will be disabled until you save your key.
        2.  **🔄 Translate:**
            *   Load a pre-built example from the dropdown or write your own Python code. The 'Translate' button is disabled if the code box is empty.
            *   Click **Translate**. The optimized C++ code will appear on the right.
        3.  **▶️ Run & Compare:**
            *   After a successful translation, click **Run Both Versions**.
            *   The tool will execute the Python code, compile and run the C++ code, and show you the output and execution times for both.
        4.  **💬 Explain:**
            *   Ask the AI assistant questions about the code, the translation, or the performance results.
        
        **System Requirements:**
        *   Python 3.10+
        *   `g++` compiler (MinGW on Windows) installed and accessible in your system's PATH.
        """)

    with gr.Tabs():
        with gr.Tab("⚙️ Settings"):
            gr.Markdown("## API Configuration")
            gr.Markdown("Get your API keys from [OpenAI](https://platform.openai.com/api-keys) or [Anthropic](https://console.anthropic.com/settings/keys). The tool securely handles your key and does not store it.")
            provider_radio = gr.Radio(["OpenAI", "Claude"], label="Select API Provider", value="OpenAI")
            api_key_textbox = gr.Textbox(label="API Key", type="password", placeholder="Enter your API key (e.g., sk-...)")
            update_button = gr.Button("Save Settings")

        with gr.Tab("🔄 Translate"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Python Input")
                    example_dropdown = gr.Dropdown(label="Load Example", choices=list(DEFAULT_PYTHON_CODE.keys()), value="Calculate Pi")
                    python_input = gr.Code(value=DEFAULT_PYTHON_CODE["Calculate Pi"], label="Python", language="python", lines=20)
                with gr.Column(scale=1):
                    gr.Markdown("### Optimized C++ Output")
                    target_language_dd = gr.Dropdown(["C++"], value="C++", label="Target Language", interactive=False)
                    cpp_output = gr.Code(label="C++", language="cpp", lines=20, interactive=False)
            translate_button = gr.Button("Translate", variant="primary", interactive=False)
            
        with gr.Tab("▶️ Run & Compare"):
            gr.Markdown("## Performance Comparison")
            run_button = gr.Button("Run Both Versions", variant="primary", interactive=False)
            results_display = gr.Textbox(label="📊 Results", lines=22, interactive=False, placeholder="Execution results and speed comparison will appear here...")

        with gr.Tab("💬 Explain"):
            gr.Markdown("## Code Explanation Chatbot")
            chatbot = gr.Chatbot(label="Code Mentor", height=550, type="messages")
            with gr.Row():
                question_box = gr.Textbox(label="Your Question", scale=4, placeholder="e.g., Why is the C++ version faster? Explain the memory allocation.")
                ask_button = gr.Button("Ask", scale=1, variant="primary", interactive=False)
            clear_button = gr.Button("Clear Chat History")

    gr.Markdown("---")
    gr.Markdown("To view code visit,https://github.com/usmanibrahim17?tab=repositories")

    # Event handlers
    update_button.click(
        update_api_settings, 
        inputs=[provider_radio, api_key_textbox, app_settings], 
        outputs=[app_settings, translate_button, run_button, ask_button], 
        queue=False
    )

    def translate_and_update_state(python_code, settings, progress=gr.Progress(track_tqdm=True)):
        print("🔥 TRANSLATE CALLED")
        print(f"Python code length: {len(python_code)}")
        translated_code = translate_code(python_code, settings, progress)
        print(f"Translated code length: {len(translated_code)}")
        clean_cpp = extract_code_from_markdown(translated_code)
        print(f"Clean C++ length: {len(clean_cpp)}")
        print(f"First 100 chars of clean C++:\n{clean_cpp[:100]}")
        return translated_code, python_code, clean_cpp

    translate_button.click(
        translate_and_update_state,
        inputs=[python_input, app_settings],
        outputs=[cpp_output, python_code_state, cpp_code_state]
    )
    
    python_input.change(
        lambda x: gr.Button(interactive=bool(x.strip())),
        inputs=[python_input],
        outputs=[translate_button],
        queue=False
    )
    
    example_dropdown.change(load_example, inputs=[example_dropdown], outputs=[python_input], queue=False)
    
    def debug_run_wrapper(py_code, cpp_code):
        print(f"=== STATE CHECK ===")
        print(f"Python code exists: {bool(py_code)}")
        print(f"Python code length: {len(py_code) if py_code else 0}")
        print(f"C++ code exists: {bool(cpp_code)}")
        print(f"C++ code length: {len(cpp_code) if cpp_code else 0}")
        
        if py_code:
            print(f"First 100 chars of Python:\n{py_code[:100]}")
        if cpp_code:
            print(f"First 100 chars of C++:\n{cpp_code[:100]}")
        
        try:
            return run_and_compare(py_code, cpp_code)
        except Exception as e:
            print(f"ACTUAL ERROR: {type(e).__name__}: {str(e)}")
            import traceback
            traceback.print_exc()
            raise
    
    run_button.click(debug_run_wrapper, inputs=[python_code_state, cpp_code_state], outputs=[results_display])
    
    ask_button.click(explain_code, inputs=[chatbot, question_box, app_settings, python_code_state, cpp_code_state], outputs=[chatbot, question_box])
    
    clear_button.click(lambda: ([], ""), outputs=[chatbot, question_box], queue=False)



In [48]:
# Launch the Gradio application
if __name__ == "__main__":
    demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Claude API key updated successfully!
